In [ ]:
file_id = "1plYmOfj3faGtNWB4EtXFBjEawbCNJb7Z"
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# PyDrive reference:
# https://googledrive.github.io/PyDrive/docs/build/html/index.html


from google.colab import auth
auth.authenticate_user()

from googleapiclient.discovery import build
drive_service = build('drive', 'v3')

# Replace the assignment below with your file ID
# to download a different file.
#
# A file ID looks like: 1gLBqEWEBQDYbKCDigHnUXNTkzl-OslSO

import io
from googleapiclient.http import MediaIoBaseDownload

request = drive_service.files().get_media(fileId=file_id)
downloaded = io.BytesIO()
downloader = MediaIoBaseDownload(downloaded, request)
done = False
while done is False:
  # _ is a placeholder for a progress object that we ignore.
  # (Our file is small, so we skip reporting progress.)
  _, done = downloader.next_chunk()

fileId = drive.CreateFile({'id': file_id }) #DRIVE_FILE_ID is file id example: 1iytA1n2z4go3uVCwE_vIKouTKyIDjEq
print(fileId['title'])  
fileId.GetContentFile(fileId['title'])  # Save Drive file as a local file

!unzip {fileId['title']}

In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import MaxPooling2D, AveragePooling2D
from tensorflow.keras.layers import Concatenate
from tensorflow.keras.layers import Lambda, Flatten, Dense
from tensorflow.keras.initializers import glorot_uniform
from tensorflow.keras.layers import Layer
from tensorflow.keras import backend as K
K.set_image_data_format('channels_last')
import os
import numpy as np
from numpy import genfromtxt
import pandas as pd
import tensorflow as tf
import PIL
import cv2
import matplotlib.pyplot as plt


In [4]:
from google.colab.patches import cv2_imshow

In [5]:
from tensorflow.keras.models import model_from_json

json_file = open('/content/model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)
model.load_weights('/content/model.h5')

In [6]:
FRmodel = model

In [7]:
def img_to_encoding(image_path, model):
    img = tf.keras.preprocessing.image.load_img(image_path, target_size=(160, 160))
    img = np.around(np.array(img) / 255.0, decimals=12)
    x_train = np.expand_dims(img, axis=0)
    embedding = model.predict_on_batch(x_train)
    return embedding / np.linalg.norm(embedding, ord=2)

In [8]:
database = {}
database["Tushar"] = img_to_encoding("/content/database/tushar.jpeg", FRmodel)
database["Harsh"] = img_to_encoding("/content/database/harsh.jpeg", FRmodel)
database["Pratyush"] = img_to_encoding("/content/database/pratyush.jpeg", FRmodel)
database["baby"] = img_to_encoding("/content/database/baby.jpeg", FRmodel)

In [9]:
def who_is_it(image_path, database, model):
    """
    Implements face recognition for the office by finding who is the person on the image_path image.
    
    Arguments:
        image_path -- path to an image
        database -- database containing image encodings along with the name of the person on the image
        model -- your Inception model instance in Keras
    
    Returns:
        min_dist -- the minimum distance between image_path encoding and the encodings from the database
        identity -- string, the name prediction for the person on image_path
    """
    
    ### START CODE HERE

    ## Step 1: Compute the target "encoding" for the image. Use img_to_encoding() see example above. ## (≈ 1 line)
    encoding =  img_to_encoding(image_path, model)
    
    ## Step 2: Find the closest encoding ##
    
    # Initialize "min_dist" to a large value, say 100 (≈1 line)
    min_dist = 100
    
    # Loop over the database dictionary's names and encodings.
    for (name, db_enc) in database.items():
        
        # Compute L2 distance between the target "encoding" and the current db_enc from the database. (≈ 1 line)
        dist = np.linalg.norm(tf.subtract(encoding, db_enc))

        # If this distance is less than the min_dist, then set min_dist to dist, and identity to name. (≈ 3 lines)
        if dist<min_dist:
            min_dist = dist
            identity = name
    ### END CODE HERE
    if min_dist > 0.77:
        return min_dist, "Intruder"
    else:
        return min_dist, str(identity)
    

**Face Recognition Program**

* A feature installed in the bot, which activates during security mode.


* Greets the people of the house, whose photos are already in the database.

* And triggers an alarm when detects an unknown person.





In [ ]:
keys=database.keys()
print(keys) #People already registered in the database.

In [ ]:
face_cascade = cv2.CascadeClassifier('/content/haarcascade_frontalface_default.xml')
img = cv2.imread('/content/test/test3.jpeg') #We have test images 1, 2, 3, 4, 5, 6.
imag=cv2.resize(img, (225,400))
cv2_imshow(imag)

In [ ]:
gray=cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
i=1
#for test image 3 the values of parameters for .detectMultiScale function should be (1.2, 5)
#for all other test images the values of parameters for .detectMultiScale function should be (1.21, 5) 
faces = face_cascade.detectMultiScale(gray, 1.21, 5)
print(len(faces))
for (x, y, w, h) in faces:
  cv2.rectangle(img, (x, y), (x+w, y+h), (255, 0, 0), 2)
  face = img[y:y+h, x:x+w]
  cv2.imwrite("Testthis"+str(i)+".jpeg", face)
  i+=1
Names= []
for i in range(1,len(faces)+1):
  _ ,names=who_is_it("/content/Testthis"+str(i)+".jpeg", database, FRmodel)
  Names.append(names)

intruderflag = 0
babyflag = 0
for name in Names:
  if (name=="Intruder"):
    intruderflag = 1
  elif(name == "baby") :
    babyflag = 1
  else:
    print("Hello "+name+"! How was your day today?")

if(intruderflag == 1 and babyflag == 1):
  print("Baby is in danger, intruder detected!! Alarm triggered")
elif(intruderflag == 1 and babyflag ==0):
  print("Intruder detected!! Alarm is triggered")

imag=cv2.resize(img, (225,400))
cv2_imshow(imag) 